In [1]:
ungapped = [
    [5, -4, -4, -4],
    [-4, 5, -4, -4],
    [-4, -4, 5, -4],
    [-4, -4, -4, 5],
]

gapped = [
    [1, -3, -3, -3],
    [-3, 1, -3, -3],
    [-3, -3, 1, -3],
    [-3, -3, -3, 1],
]

blosum62 = [
    [4, 0, -2, -1, -2, 0, -2, -1, -1, -1, -1, -2, -1, -1, -1, 1, 0, 0, -3, -2],
    [0, 9, -3, -4, -2, -3, -3, -1, -3, -1, -1, -3, -3, -3, -3, -1, -1, -1, -2, -2],
    [-2, -3, 6, 2, -3, -1, -1, -3, -1, -4, -3, 1, -1, 0, -2, 0, -1, -3, -4, -3],
    [-1, -4, 2, 5, -3, -2, 0, -3, 1, -3, -2, 0, -1, 2, 0, 0, -1, -2, -3, -2],
    [-2, -2, -3, -3, 6, -3, -1, 0, -3, 0, 0, -3, -4, -3, -3, -2, -2, -1, 1, 3],
    [0, -3, -1, -2, -3, 6, -2, -4, -2, -4, -3, 0, -2, -2, -2, 0, -2, -3, -2, -3],
    [-2, -3, -1, 0, -1, -2, 8, -3, -1, -3, -2, 1, -2, 0, 0, -1, -2, -3, -2, 2],
    [-1, -1, -3, -3, 0, -4, -3, 4, -3, 2, 1, -3, -3, -3, -3, -2, -1, 3, -3, -1],
    [-1, -3, -1, 1, -3, -2, -1, -3, 5, -2, -1, 0, -1, 1, 2, 0, -1, -2, -3, -2],
    [-1, -1, -4, -3, 0, -4, -3, 2, -2, 4, 2, -3, -3, -2, -2, -2, -1, 1, -2, -1],
    [-1, -1, -3, -2, 0, -3, -2, 1, -1, 2, 5, -2, -2, 0, -1, -1, -1, 1, -1, -1],
    [-2, -3, 1, 0, -3, 0, 1, -3, 0, -3, -2, 6, -2, 0, 0, 1, 0, -3, -4, -2],
    [-1, -3, -1, -1, -4, -2, -2, -3, -1, -3, -2, -2, 7, -1, -2, -1, -1, -2, -4, -3],
    [-1, -3, 0, 2, -3, -2, 0, -3, 1, -2, 0, 0, -1, 5, 1, 0, -1, -2, -2, -1],
    [-1, -3, -2, 0, -3, -2, 0, -3, 2, -2, -1, 0, -2, 1, 5, -1, 0, -3, -3, -2],
    [1, -1, 0, 0, -2, 0, -1, -2, 0, -2, -1, 1, -1, 0, -1, 4, 1, -2, -3, -2],
    [0, -1, -1, -1, -2, -2, -2, -1, -1, -1, -1, 0, -1, -1, -1, 1, 5, 0, -2, -2],
    [0, -1, -3, -2, -1, -3, -3, 3, -2, 1, 1, -3, -2, -2, -3, -2, 0, 4, -3, -1],
    [-3, -2, -4, -3, 1, -2, -2, -3, -3, -2, -1, -4, -4, -2, -3, -3, -2, -3, 11, 2],
    [-2, -2, -3, -2, 3, -3, 2, -1, -2, -1, -1, -2, -3, -1, -2, -2, -2, -1, 2, 7],
]

nucleotide = ['A', 'T', 'C', 'G']
amino_acid = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']

In [8]:
import numpy as np
with open('spike_cov_nuc.txt', 'r') as file:
    seq_1 = file.read().replace('\n', '')
with open('spike_cov2_nuc.txt', 'r') as file:
    seq_2 = file.read().replace('\n', '')
n = len(seq_1)
m = len(seq_2)
scoring_matrix = np.zeros((n, m))
gap = 3
for i in range(n):
    for j in range(m):
        first = nucleotide.index(seq_1[i])
        second = nucleotide.index(seq_2[j])
        scoring_matrix[i][j] = ungapped[first][second]
# print("Score for each combination of bases in both sequences:")
# print(scoring_matrix)
# print("\n\n")
algo_matrix = np.zeros((n + 1, m + 1))
# for i in range(n + 1):
#     algo_matrix[i][0] = -i * gap
# for j in range(m + 1):
#     algo_matrix[0][j] = -j * gap
# print(algo_matrix)
for i in range(1, n + 1):
    for j in range(1, m + 1):
        a = algo_matrix[i - 1][j - 1] + scoring_matrix[i - 1][j - 1]
        b = algo_matrix[i - 1][j] - gap
        c = algo_matrix[i][j - 1] - gap
        algo_matrix[i][j] = max(0, a, b, c)
# print("Smith-Waterman Matrix:")
# print(algo_matrix)
# print("\n\n")
aligned_seq_1 = ""
aligned_seq_2 = ""
i = n
j = m
while (i > 0 or j > 0):
    cell_score = algo_matrix[i][j]
    if int(cell_score) == 0:
        break
    diag_score = algo_matrix[i - 1][j - 1]
#     above_score = algo_matrix[i][j - 1]
    left_score = algo_matrix[i - 1][j]
    if cell_score == diag_score + scoring_matrix[i - 1][j - 1]:
        aligned_seq_1 += seq_1[i - 1]
        aligned_seq_2 += seq_2[j - 1]
        i -= 1
        j -= 1
    elif cell_score == left_score - gap:
        aligned_seq_1 += seq_1[i - 1]
        aligned_seq_2 += "_"
        i -= 1
    else:
        aligned_seq_1 += "_"
        aligned_seq_2 += seq_2[j - 1]
        j -= 1
aligned_seq_1 = aligned_seq_1[::-1]
aligned_seq_2 = aligned_seq_2[::-1]
print("The aligned sequences:")
print(aligned_seq_1)
print(aligned_seq_2)

The aligned sequences:
ATGTTT_ATTTTC_T_TATTATTTCTTACT_CTCACTAGT_GGTAGTGACCTTGACCGGTGC_ACCACTTTTG_A_TGATGTTCAAGCTCCT_AATTACACTCAACATAC_TTCATCTATGAGGGGGGTTTACTATCCTGATGAAA_TTTTTAGATCAGACACTCTTT__ATTTAACTCAGGATTTATTTCTT_CCATTT_TATTCTAATGTTACAGGGTT___T_C_ATAC___TAT_T______AA___T_C__ATACGTTTGGCAACCCTGTCATACCTTTTAAGGATGGTATTTATTTTGCTGCCACAGAGAAATC_AA_ATGTTGTCCGTGGTTGGGTTTTTGGTTCTACCAT_GA_ACAACAAGTCACAGTCGGTGA_TTATTATTAACAATTCTACTAATGTTGTTA_TACGAG_CATGT_AACTTTGAATTGTGTGACA__A_CCCTTTCTTTGCTGTTTCTAAACC_CATGGGTACACAGACACATA_CT__AT_G__A_T_A_TTC_GA___TA____A_TGC_ATTTAATTGCACTTTCGAGTACATATCTGATGCCTTTTCGCTTGAT_G___TTTCA_GAAAAGTCA_GGTAATTTTAAACA_CTTACGAGAGTTTGTGTTTAAAAATAAAGATGGGTT_TCTCT___ATGT_TTATAAGGGCTATCA_ACCTA_TAGATGTAGTTCGTGATCTACCTTC_TGGTTTTAAC_ACTTT_GAAACCTATTTTTA_AGTTGCCTCTTGGTATTAACAT___TA_____CAAATTTTAGAGCCATT_C_TTAC__AG__CCT_TTT__CACCTGCTCAAGA__CAT_TT__GG__GG_CA_C__GTCAGCTGCAGCCTATTTTGTTGGCTAT_TTAAAGCC_A__ACTACATT_TATGCTCAAGTATGATGAAAATGGTACAATCACAGATGCTGTTGATTGTTCTCAAAATCCACT_TGCT__GAAC

# checking with emboss water
                   1 ATGTTT-ATTTTCTTATTATTTCTTACTCTCACTAGT-GGTAGTGACCTT     48
                     |||||| .|||||||.||     |||.|..||||||| ..||||      
                   1 ATGTTTGTTTTTCTTGTT-----TTATTGCCACTAGTCTCTAGT------     39

                  49 GACCGGTGCACCACTTTTGATGATGTT-----CA-AGCTCCTAATTACAC     92
                        |.|||         ||.|.||.||     || |.|||  ||||||.|
                  40 ---CAGTG---------TGTTAATCTTACAACCAGAACTC--AATTACCC     75

                  93 TCAACATAC-----TTCATCTATGAGGGGGGTTTACTATCCTGATGAAAT    137
                     .|..|||||     |||.|..|...|.||.|||||.||.|||||..||.|
                  76 CCTGCATACACTAATTCTTTCACACGTGGTGTTTATTACCCTGACAAAGT    125

                 138 TTTTAGATCAGACACTC---TTT--ATTTAACTCAGGATTTATTTCTT-C    181
                     |||.|||||     |||   |||  |||.|||||||||.|| .||||| |
                 126 TTTCAGATC-----CTCAGTTTTACATTCAACTCAGGACTT-GTTCTTAC    169

                 182 CATTTTATTCTAATGTTACAGGGTTTCATACTAT----------------    215
                     |.||.|.|||.||||||||..||||.|||.||||                
                 170 CTTTCTTTTCCAATGTTACTTGGTTCCATGCTATACATGTCTCTGGGACC    219

                 216 -----TAATCATACGTTTGGCAACCCTGTCATACCTTTTAAGGATGGTAT    260
                          ||.|.|.|.|||||..|||||||||.||||.|||||.||||||.|
                 220 AATGGTACTAAGAGGTTTGATAACCCTGTCCTACCATTTAATGATGGTGT    269

                 261 TTATTTTGCTGCCACAGAGAAATCAAATGTTGTCCGTGGTTGGGTTTTTG    310
                     ||||||||||.||||.|||||.||.||..|..|..|.||.|||.||||||
                 270 TTATTTTGCTTCCACTGAGAAGTCTAACATAATAAGAGGCTGGATTTTTG    319

                 311 GTTCTACCATGAACAACAAGTCACAGTCGGTGATTATTATTAACAATTCT    360
                     ||.||||..|..|....|||.|.|||||..|..|||||.||||.||..||
                 320 GTACTACTTTAGATTCGAAGACCCAGTCCCTACTTATTGTTAATAACGCT    369

                 361 ACTAATGTTGTTATACGAGCATGTAACTTTGAATTGTGTGACAACCCTTT    410
                     ||||||||||||||...||..|||.|.|||.||||.|||.|..|.||.||
                 370 ACTAATGTTGTTATTAAAGTCTGTGAATTTCAATTTTGTAATGATCCATT    419

                 411 CTTTGCTGTTTCTAAACCCATGGGTA-CACAGACACATACTA--------    451
                     .||.|.|||||.|           || ||||.|.||| ||.|        
                 420 TTTGGGTGTTTAT-----------TACCACAAAAACA-ACAAAAGTTGGA    457

                 452 -------TGA----------TATTCGATAATGCATTTAATTGCACTTTCG    484
                            |||          |||||  ||.|||...||||||||||||.|
                 458 TGGAAAGTGAGTTCAGAGTTTATTC--TAGTGCGAATAATTGCACTTTTG    505

                 485 AGTACATATCTGATGCCTTTTC-------GCTTGATGTTTCAGAAAAGTC    527
                     |.||..|.|||.| ||||||||       .|||||.|     |||||  |
                 506 AATATGTCTCTCA-GCCTTTTCTTATGGACCTTGAAG-----GAAAA--C    547

                 528 A-GGTAATTTTAAACA-CTTACGAGAGTTTGTGTTTAAAAATAAAGATGG    575
                     | ||||||||.|||.| |||| |.||.|||||||||||.||||..|||||
                 548 AGGGTAATTTCAAAAATCTTA-GGGAATTTGTGTTTAAGAATATTGATGG    596

                 576 GTTTCTCTATGTTTATAAGGGCTAT----------CA-ACCTATAGATGT    614
                     .|.|       ||||.||    |||          || .|||||..||.|
                 597 TTAT-------TTTAAAA----TATATTCTAAGCACACGCCTATTAATTT    635

                 615 AGTTCGTGATCTACCTTCTGGTTTTAACACTTTGAAACCTATTTTTA-AG    663
                     |||.||||||||.|||...||||||....||||..|||| |||..|| |.
                 636 AGTGCGTGATCTCCCTCAGGGTTTTTCGGCTTTAGAACC-ATTGGTAGAT    684

                 664 TTGCCTCTTGGTATTAACA--------TTACAAATTTTAGAGCCATTC-T    704
                     |||||..|.||||||||||        ||.||||.||||    |.|.| |
                 685 TTGCCAATAGGTATTAACATCACTAGGTTTCAAACTTTA----CTTGCTT    730

                 705 TAC-----AGCCTTTT--CACCTG----------CTCAAGACATTTGG--    735
                     |||     ||...|||  |.||||          .|||.|    ||||  
                 731 TACATAGAAGTTATTTGACTCCTGGTGATTCTTCTTCAGG----TTGGAC    776

                 736 GGCACGTCAGCTGCAGCCTATTTTGTTGGCTAT-TTAAAGCCAACTACAT    784
                     .||..||  ||||||||.||||.|||.||.||| ||.|| ||.|..||.|
                 777 AGCTGGT--GCTGCAGCTTATTATGTGGGTTATCTTCAA-CCTAGGACTT    823

                 785 TTATGCTCAAGTATGATGAAAATGGTACAATCACAGATGCTGTTGATTGT    834
                     ||.|..|.||.|||.||||||||||.||.||.|||||||||||.||.|||
                 824 TTCTATTAAAATATAATGAAAATGGAACCATTACAGATGCTGTAGACTGT    873

                 835 TCTCAAAATCCACTTGCTGAACTCAAA-TGCTCTGTTAAGAGCTTTGA--    881
                     .|.|...|.||.||..|.||| .|||| || |..|||.|.|.|.||.|  
                 874 GCACTTGACCCTCTCTCAGAA-ACAAAGTG-TACGTTGAAATCCTTCACT    921

                 882 GATTGACAAAGGAATTTACCAGACCTCTAATTTCAGGGTTGTTCCCTCAG    931
                     | |.||.||||||||.||.||.||.|||||.||.||.||....||..|||
                 922 G-TAGAAAAAGGAATCTATCAAACTTCTAACTTTAGAGTCCAACCAACAG    970

                 932 GAGATGTTGTGAGATTCCCTAATATTACAAACTTGTGTCCTTTTGGAGAG    981
                     .|..|.||||.|||||.||||||||||||||||||||.||||||||.||.
                 971 AATCTATTGTTAGATTTCCTAATATTACAAACTTGTGCCCTTTTGGTGAA   1020

                 982 GTTTTTAATGCTACTAAATTCCCTTCTGTCTATGCATGGGAGAGAAAAAA   1031
                     ||||||||.||.||.|.|||..|.|||||.|||||.|||.|.||.||.|.
                1021 GTTTTTAACGCCACCAGATTTGCATCTGTTTATGCTTGGAACAGGAAGAG   1070

                1032 AATTTCTAATTGTGTTGCTGATTACTCTGTGCTCTACAACTCAACATTTT   1081
                     |||....||.||||||||||||||.|||||.||.||.||.||..|||..|
                1071 AATCAGCAACTGTGTTGCTGATTATTCTGTCCTATATAATTCCGCATCAT   1120

                1082 TTTCAACCTTTAAGTGCTATGGCGTTTCTGCCACTAAGTTGAATGATCTT   1131
                     ||||.||.||||||||.|||||.||.|||.|.|||||.||.||||||||.
                1121 TTTCCACTTTTAAGTGTTATGGAGTGTCTCCTACTAAATTAAATGATCTC   1170

                1132 TGCTTCTCCAATGTCTATGCAGATTCTTTTGTAGTCAAGGGAGATGATGT   1181
                     |||||..|.|||||||||||||||||.||||||.|.|..||.|||||.||
                1171 TGCTTTACTAATGTCTATGCAGATTCATTTGTAATTAGAGGTGATGAAGT   1220

                1182 AAGACAAATAGCGCCAGGACAAACTGGTGTTATTGCTGATTATAATTATA   1231
                     .||||||||.||.|||||.||||||||....|||||||||||||||||||
                1221 CAGACAAATCGCTCCAGGGCAAACTGGAAAGATTGCTGATTATAATTATA   1270

                1232 AATTGCCAGATGATTTCATGGGTTGTGTCCTTGCTTGGAATACTAGGAAC   1281
                     ||||.|||||||||||.|..||.||.||..|.|||||||||.|||..||.
                1271 AATTACCAGATGATTTTACAGGCTGCGTTATAGCTTGGAATTCTAACAAT   1320

                1282 ATTGATGCTA--CTTCAACTGGTAATTATAATTATAAATATAGGTATCTT   1329
                     .|||||.|||  .||  ..|||||||||||||||....|||||.|...||
                1321 CTTGATTCTAAGGTT--GGTGGTAATTATAATTACCTGTATAGATTGTTT   1368

                1330 AGACATGGC-AAGCTTAGGCCCTTTGAGAGAGACATATCTAATG---TGC   1375
                     || .|.|.| ||.||.|..||.|||||||||||.||.||.|.||   |  
                1369 AG-GAAGTCTAATCTCAAACCTTTTGAGAGAGATATTTCAACTGAAAT--   1415

                1376 CTTTCTCCCCTGATGGCAAACCTTGCACCCCACCT---------GCTCTT   1416
                     ||.||...|| |.|.|||.||||||.|      .|         |.|.||
                1416 CTATCAGGCC-GGTAGCACACCTTGTA------ATGGTGTTGAAGGTTTT   1458

                1417 AATTGTTATTGGCCATTA-AATGATTATGGTTTTTACACCACTACTGGCA   1465
                     ||||||||.|..||.||| |||.| ||||||||..|..||||||.|||..
                1459 AATTGTTACTTTCCTTTACAATCA-TATGGTTTCCAACCCACTAATGGTG   1507

                1466 TTGGCTACCAACCTTACAGAGTTGTAGTACTTTCTTTTGAACTTTTAAAT   1515
                     ||||.||||||||.||||||||.|||||||||||||||||||||.||.||
                1508 TTGGTTACCAACCATACAGAGTAGTAGTACTTTCTTTTGAACTTCTACAT   1557

                1516 GCACCGGCCACGGTTTGTGGACCAAAATTATCCACTGACCTTATTAAGAA   1565
                     |||||.||.||.|||||||||||.|||...||.|||.|..|..||||.||
                1558 GCACCAGCAACTGTTTGTGGACCTAAAAAGTCTACTAATTTGGTTAAAAA   1607

                1566 CCAGTGTGTCAATTTTAATTTTAATGGACTCACTGGTACTGGTGTGTTAA   1615
                     |.|.|||||||||||.||.||.|||||..|.||.||.||.|||||..|.|
                1608 CAAATGTGTCAATTTCAACTTCAATGGTTTAACAGGCACAGGTGTTCTTA   1657

                1616 CTCCTTCT-TCAAAGAGATTTCAACCATTTCAACAATTTGGCCGTGATGT   1664
                     ||...||| .||||.|| ||||..||.||.||||||||||||.|.||..|
                1658 CTGAGTCTAACAAAAAG-TTTCTGCCTTTCCAACAATTTGGCAGAGACAT   1706

                1665 TTCTGATTTCACTGATTCCGTTCGAGATCCTAAAACATCTGAAATATTAG   1714
                     |.||||....||||||.|.||.||.|||||..|.|||..|||.||..|.|
                1707 TGCTGACACTACTGATGCTGTCCGTGATCCACAGACACTTGAGATTCTTG   1756

                1715 ACATTTCACCTTGCGCTTTTGGGGGTGTAAGTGTAATTACACCTGGAACA   1764
                     |||||.||||.||..|||||||.|||||.|||||.||.|||||.||||||
                1757 ACATTACACCATGTTCTTTTGGTGGTGTCAGTGTTATAACACCAGGAACA   1806

                1765 AATGCTTC--ATCTGAAGTTGCTGTTCTATATCAAGATGTTAACTGCACT   1812
                     |||.||||  |.|  |.|||||||||||.|||||.||||||||||||||.
                1807 AATACTTCTAACC--AGGTTGCTGTTCTTTATCAGGATGTTAACTGCACA   1854

                1813 GATGTTTCTACAGCAATTCATGCAGATCAACTCACACCAGCTTGGCGCAT   1862
                     ||.||..||...||.|||||||||||||||||.||.||..|||||||..|
                1855 GAAGTCCCTGTTGCTATTCATGCAGATCAACTTACTCCTACTTGGCGTGT   1904

                1863 ATATTCTACTGGAAACAATGTATTCCAGACTCAAGCAGGCTGTCTTATAG   1912
                     .||||||||.||....|||||.||.||.||.|..|||||||||.|.||||
                1905 TTATTCTACAGGTTCTAATGTTTTTCAAACACGTGCAGGCTGTTTAATAG   1954

                1913 GAGCTGAGCATGTCGACACTTCTTATGAGTGCGACATTCCTATTGGAGCT   1962
                     |.|||||.||||||.|||..||.||||||||.|||||.||.|||||.||.
                1955 GGGCTGAACATGTCAACAACTCATATGAGTGTGACATACCCATTGGTGCA   2004

                1963 GGCATTTGTGCTAGTTACCATACAGTTTC---TTTATT------------   1997
                     ||.||.||.||||||||.||.||    ||   .|.|||            
                2005 GGTATATGCGCTAGTTATCAGAC----TCAGACTAATTCTCCTCGGCGGG   2050

                1998 -ACGTAGT---ACTAGCCAAAAATCTATTGTGGCTTATACTATGTCTTTA   2043
                      |||||||   .||||.|   ||||.||..|.||.||.||||||||..|.
                2051 CACGTAGTGTAGCTAGTC---AATCCATCATTGCCTACACTATGTCACTT   2097

                2044 GGTGCTGATAGTTCAATTGCTTACTCTAATAACACCATTGCTATACCTAC   2093
                     |||||.||.|.||||.|||||||||||||||||.|.|||||.|||||.||
                2098 GGTGCAGAAAATTCAGTTGCTTACTCTAATAACTCTATTGCCATACCCAC   2147

                2094 TAACTTTTCAATTAGCATTACTACAGAAGTAATGCCTGTTTCTATGGCTA   2143
                     .||.|||.|.|||||..||||.||||||.|..|.||.||.||||||.|.|
                2148 AAATTTTACTATTAGTGTTACCACAGAAATTCTACCAGTGTCTATGACCA   2197

                2144 AAACCTCCGTAGATTGTAATATGTACATCTGCGGAGATTCTACTGAATG-   2192
                     |.||.||.||||||||||..||||||||.||.||.|||||.|||||||| 
                2198 AGACATCAGTAGATTGTACAATGTACATTTGTGGTGATTCAACTGAATGC   2247

                2193 TGCTAATTTGCTTCTCCAATATGGTAGCTTTTGCACACAACTAAATCGTG   2242
                     .|| |||.|..|..|.||||||||.||.|||||.||||||.||||.||||
                2248 AGC-AATCTTTTGTTGCAATATGGCAGTTTTTGTACACAATTAAACCGTG   2296

                2243 C----ACTCTCAGGTATTGCTGCTGAACAGGATCGCAACACACGTGAAGT   2288
                     |    |||    ||.||.||||.||||||.||....|||||.|..|||||
                2297 CTTTAACT----GGAATAGCTGTTGAACAAGACAAAAACACCCAAGAAGT   2342

                2289 GTTCGCTCAAGTCAAACAAATGTACAAAACCCCAACTTTGAAATATTTTG   2338
                     .||.||.||||||||||||||.||||||||.|||.|..|.|||.||||||
                2343 TTTTGCACAAGTCAAACAAATTTACAAAACACCACCAATTAAAGATTTTG   2392

                2339 GTGGTTTTAATTTTTCACAAATATTACCTGA-CCCTCTAAAGCCAACTAA   2387
                     ||||||||||||||||||||||||||||.|| ||.|| |||.||||..||
                2393 GTGGTTTTAATTTTTCACAAATATTACCAGATCCATC-AAAACCAAGCAA   2441

                2388 GAGGTCTTTTATTGAGGACTTGCTCTTTAATAAGGTGACACTCGCTGATG   2437
                     ||||||.||||||||.||..|.||.||.||.||.||||||||.||.||||
                2442 GAGGTCATTTATTGAAGATCTACTTTTCAACAAAGTGACACTTGCAGATG   2491

                2438 CTGGCTTCATGAAGCAATATGGCGAATGCCTAGGTGATATTAATGCTAGA   2487
                     ||||||||||.||.||||||||.||.|||||.|||||||||..|||||||
                2492 CTGGCTTCATCAAACAATATGGTGATTGCCTTGGTGATATTGCTGCTAGA   2541

                2488 GATCTCATTTGTGCGCAGAAGTTCAATGGACTTACAGTGTTGCCACCTCT   2537
                     ||.|||||||||||.||.|||||.||.||.|||||.||.|||||||||.|
                2542 GACCTCATTTGTGCACAAAAGTTTAACGGCCTTACTGTTTTGCCACCTTT   2591

                2538 GCTCACTGATGATATGATTGCTGCCTACACTGCTGCTCTAGTTAG-TGGT   2586
                     ||||||.|||||.|||||||||...||||||.||||.|| ||||| .|||
                2592 GCTCACAGATGAAATGATTGCTCAATACACTTCTGCACT-GTTAGCGGGT   2640

                2587 ACTGCCACTGCTGGATGGACATTTGGTGCTGGCGCTGCTCTTCAAATACC   2636
                     ||...||||.||||.|||||.||||||||.||.|||||..|.||||||||
                2641 ACAATCACTTCTGGTTGGACCTTTGGTGCAGGTGCTGCATTACAAATACC   2690

                2637 TTTTGCTATGCAAATGGCATATAGGTTCAATGGCATTGGAGTTACCCAAA   2686
                     .|||||||||||||||||.||||||||.|||||.|||||||||||.||.|
                2691 ATTTGCTATGCAAATGGCTTATAGGTTTAATGGTATTGGAGTTACACAGA   2740

                2687 ATGTTCTCTATGAGAACCAAAAACAAATCGCCAACCAATTTAACAAGGCG   2736
                     |||||||||||||||||||||||...||.||||||||||||||.|..||.
                2741 ATGTTCTCTATGAGAACCAAAAATTGATTGCCAACCAATTTAATAGTGCT   2790

                2737 ATTAGTCAAATTCAAGAATCACTTACAACAACATCAACTGCATTGGGCAA   2786
                     |||.|..||||||||||.||||||.|..|.|||.|||.||||.|.||.||
                2791 ATTGGCAAAATTCAAGACTCACTTTCTTCCACAGCAAGTGCACTTGGAAA   2840

                2787 GCTGCAAGACGTTGTTAACCAGAATGCTCAAGCATTAAACACACTTGTTA   2836
                     .||.|||||.||.||.|||||.|||||.|||||.||||||||.|||||||
                2841 ACTTCAAGATGTGGTCAACCAAAATGCACAAGCTTTAAACACGCTTGTTA   2890

                2837 AACAACTTAGCTCTAATTTTGGTGCAATTTCAAGTGTGCTAAATGATATC   2886
                     |||||||||||||.|||||||||||||||||||||||..|||||||||||
                2891 AACAACTTAGCTCCAATTTTGGTGCAATTTCAAGTGTTTTAAATGATATC   2940

                2887 CTTTCGCGACTTGATAAAGTCGAGGCGGAGGTACAAATTGACAGGTTAAT   2936
                     |||||.||.|||||.|||||.|||||.||.||.||||||||.|||||.||
                2941 CTTTCACGTCTTGACAAAGTTGAGGCTGAAGTGCAAATTGATAGGTTGAT   2990

                2937 TACAGGCAGACTTCAAAGCCTTCAAACCTATGTAACACAACAACTAATCA   2986
                     .|||||||||||||||||..|.||.||.|||||.||.||||||.||||.|
                2991 CACAGGCAGACTTCAAAGTTTGCAGACATATGTGACTCAACAATTAATTA   3040

                2987 GGGCTGCTGAAATCAGGGCTTCTGCTAATCTTGCTGCTACTAAAATGTCT   3036
                     |.|||||.||||||||.||||||||||||||||||||||||||||||||.
                3041 GAGCTGCAGAAATCAGAGCTTCTGCTAATCTTGCTGCTACTAAAATGTCA   3090

                3037 GAGTGTGTTCTTGGACAATCAAAAAGAGTTGACTTTTGTGGAAAGGGCTA   3086
                     ||||||||.|||||||||||||||||||||||.|||||||||||||||||
                3091 GAGTGTGTACTTGGACAATCAAAAAGAGTTGATTTTTGTGGAAAGGGCTA   3140

                3087 CCACCTTATGTCCTTCCCACAAGCAGCCCCGCATGGTGTTGTCTTCCTAC   3136
                     .||.||||||||||||||.||..||||.||.||||||||.||||||.|.|
                3141 TCATCTTATGTCCTTCCCTCAGTCAGCACCTCATGGTGTAGTCTTCTTGC   3190

                3137 ATGTCACGTATGTGCCATCCCAGGAGAGGAACTTCACCACAGCGCCAGCA   3186
                     ||||.||.|||||.||..|.||.||.|.|||||||||.||.||.||.||.
                3191 ATGTGACTTATGTCCCTGCACAAGAAAAGAACTTCACAACTGCTCCTGCC   3240

                3187 ATTTGTCATGAAGGCAAAGCATACTTCCCTCGTGAAGGTGTTTTTGTGTT   3236
                     |||||||||||.||.||||||.||||.||||||||||||||.|||||.|.
                3241 ATTTGTCATGATGGAAAAGCACACTTTCCTCGTGAAGGTGTCTTTGTTTC   3290

                3237 TAATGGCACTTCTTGGTTTATTACACAGAGGAACTTCTTTTCTCCACAAA   3286
                     .||||||||....||||||.|.|||||.|||||.||.|.|...|||||||
                3291 AAATGGCACACACTGGTTTGTAACACAAAGGAATTTTTATGAACCACAAA   3340

                3287 TAATTACTACAGACAATACATTTGTCTCAGGAAATTGTGATGTCGTTATT   3336
                     |.||||||||||||||.||||||||.||.||.||.||||||||.||.||.
                3341 TCATTACTACAGACAACACATTTGTGTCTGGTAACTGTGATGTTGTAATA   3390

                3337 GGCATCATTAACAACACAGTTTATGATCCTCTGCAACCTGAGCTTGACTC   3386
                     ||.||..|.|||||||||||||||||||||.||||||||||..|.|||||
                3391 GGAATTGTCAACAACACAGTTTATGATCCTTTGCAACCTGAATTAGACTC   3440

                3387 ATTCAAAGAAGAGCTGGACAAGTACTTCAAAAATCATACATCACCAGATG   3436
                     ||||||.||.|||.|.||.||.||.||.||.|||||||||||||||||||
                3441 ATTCAAGGAGGAGTTAGATAAATATTTTAAGAATCATACATCACCAGATG   3490

                3437 TTGATCTTGGCGACATTTCAGGCATTAACGCTTCTGTCGTCAACATTCAA   3486
                     |||||.|.||.|||||.||.||||||||.|||||.||.||.|||||||||
                3491 TTGATTTAGGTGACATCTCTGGCATTAATGCTTCAGTTGTAAACATTCAA   3540

                3487 AAAGAAATTGACCGCCTCAATGAGGTCGCTAAAAATTTAAATGAATCACT   3536
                     ||||||||||||||||||||||||||.||.||.||||||||||||||.||
                3541 AAAGAAATTGACCGCCTCAATGAGGTTGCCAAGAATTTAAATGAATCTCT   3590

                3537 CATTGACCTTCAAGAATTGGGAAAATATGAGCAATATATTAAATGGCCTT   3586
                     |||.||.||.||||||.|.|||||.||||||||.|||||.||||||||.|
                3591 CATCGATCTCCAAGAACTTGGAAAGTATGAGCAGTATATAAAATGGCCAT   3640

                3587 GGTATGTTTGGCTCGGCTTCATTGCTGGACTAATTGCCATCGTCATGGTT   3636
                     ||||..|||||||.||.||.||.|||||..|.||||||||.||.|||||.
                3641 GGTACATTTGGCTAGGTTTTATAGCTGGCTTGATTGCCATAGTAATGGTG   3690

                3637 ACAATCTTGCTTTGTTGCATGACTAGTTGTTGCAGTTGCCTCAAGGG-TG   3685
                     |||||..|||||||.||.|||||.|||||.||.|||||.|||||||| ||
                3691 ACAATTATGCTTTGCTGTATGACCAGTTGCTGTAGTTGTCTCAAGGGCTG   3740

                3686 CATGCTCTTGTGGTTCTTGCTGCAAGTTTGATGAGGATGACTCTGAGCCA   3735
                      .||.||||||||.||.||||||||.||||||||.||.||||||||||||
                3741 -TTGTTCTTGTGGATCCTGCTGCAAATTTGATGAAGACGACTCTGAGCCA   3789

                3736 GTTCTCAAGGGTGTCAAATTACATTACACATAA   3768
                     ||.|||||.||.|||||||||||||||||||||
                3790 GTGCTCAAAGGAGTCAAATTACATTACACATAA   3822


In [4]:
import numpy as np
with open('a.txt', 'r') as file:
    seq_1 = file.read().replace('\n', '')
with open('c.txt', 'r') as file:
    seq_2 = file.read().replace('\n', '')
n = len(seq_1)
m = len(seq_2)
scoring_matrix = np.zeros((n, m))
gap = 11
for i in range(n):
    for j in range(m):
        first = amino_acid.index(seq_1[i])
        second = amino_acid.index(seq_2[j])
        scoring_matrix[i][j] = blosum62[first][second]
# print("Score for each combination of bases in both sequences:")
# print(scoring_matrix)
# print("\n\n")
algo_matrix = np.zeros((n + 1, m + 1))
# for i in range(n + 1):
#     algo_matrix[i][0] = -i * gap
# for j in range(m + 1):
#     algo_matrix[0][j] = -j * gap
# print(algo_matrix)
for i in range(1, n + 1):
    for j in range(1, m + 1):
        a = algo_matrix[i - 1][j - 1] + scoring_matrix[i - 1][j - 1]
        b = algo_matrix[i - 1][j] - gap
        c = algo_matrix[i][j - 1] - gap
        algo_matrix[i][j] = max(0, a, b, c)
# print("Smith-Waterman Matrix:")
# print(algo_matrix)
# print("\n\n")
aligned_seq_1 = ""
aligned_seq_2 = ""
i = n
j = m
while (i > 0 or j > 0):
    cell_score = algo_matrix[i][j]
    if cell_score == 0:
        break
    diag_score = algo_matrix[i - 1][j - 1]
#     above_score = algo_matrix[i][j - 1]
    left_score = algo_matrix[i - 1][j]
    if cell_score == diag_score + scoring_matrix[i - 1][j - 1]:
        aligned_seq_1 += seq_1[i - 1]
        aligned_seq_2 += seq_2[j - 1]
        i -= 1
        j -= 1
    elif cell_score == left_score - gap:
        aligned_seq_1 += seq_1[i - 1]
        aligned_seq_2 += "_"
        i -= 1
    else:
        aligned_seq_1 += "_"
        aligned_seq_2 += seq_2[j - 1]
        j -= 1
aligned_seq_1 = aligned_seq_1[::-1]
aligned_seq_2 = aligned_seq_2[::-1]
print("The aligned sequences:")
print(aligned_seq_1)
print(aligned_seq_2)

The aligned sequences:
QCADGSCIHGSRQ
QCDDGSCIHG___


# checking with emboss water

                   1 MFIFLLFLTLTSGSDLDRCTTFDDVQAPNYTQHTSSMRGVYYPDEIFRSD     50
                     ||:||:.|.|.| |.....||...: .|.||  .|..|||||||::|||.
                   1 MFVFLVLLPLVS-SQCVNLTTRTQL-PPAYT--NSFTRGVYYPDKVFRSS     46

                  51 TLYLTQDLFLPFYSNVTGFHTI-----NHT--FGNPVIPFKDGIYFAATE     93
                     .|:.||||||||:||||.||.|     |.|  |.|||:||.||:|||:||
                  47 VLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTE     96

                  94 KSNVVRGWVFGSTMNNKSQSVIIINNSTNVVIRACNFELCDNPFFAV---    140
                     |||::|||:||:|:::|:||::|:||:|||||:.|.|:.|::||..|   
                  97 KSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYH    146

                 141 -SKPMGTQTHTMIFDNAFNCTFEYISDAFSLDVSEKSGNFKHLREFVFKN    189
                      :.....::...::.:|.||||||:|..|.:|:..|.||||:||||||||
                 147 KNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKN    196

                 190 KDGFLYVYKGYQPIDVVRDLPSGFNTLKPIFKLPLGINITNFRAIL----    235
                     .||:..:|..:.||::|||||.||:.|:|:..||:|||||.|:.:|    
                 197 IDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHR    246

                 236 TAFSPAQDI--WGTSAAAYFVGYLKPTTFMLKYDENGTITDAVDCSQNPL    283
                     :..:|....  |...||||:||||:|.||:|||:|||||||||||:.:||
                 247 SYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPL    296

                 284 AELKCSVKSFEIDKGIYQTSNFRVVPSGDVVRFPNITNLCPFGEVFNATK    333
                     :|.||::|||.::|||||||||||.|:..:|||||||||||||||||||:
                 297 SETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATR    346

                 334 FPSVYAWERKKISNCVADYSVLYNSTFFSTFKCYGVSATKLNDLCFSNVY    383
                     |.|||||.||:||||||||||||||..||||||||||.||||||||:|||
                 347 FASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVY    396

                 384 ADSFVVKGDDVRQIAPGQTGVIADYNYKLPDDFMGCVLAWNTRNIDATST    433
                     |||||::||:||||||||||.||||||||||||.|||:|||:.|:|:...
                 397 ADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVG    446

                 434 GNYNYKYRYLRHGKLRPFERDISNVPFSPDGKPCT-PPALNCYWPLNDYG    482
                     |||||.||..|...|:|||||||...:.....||. ....|||:||..||
                 447 GNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYG    496

                 483 FYTTTGIGYQPYRVVVLSFELLNAPATVCGPKLSTDLIKNQCVNFNFNGL    532
                     |..|.|:|||||||||||||||:|||||||||.||:|:||:|||||||||
                 497 FQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGL    546

                 533 TGTGVLTPSSKRFQPFQQFGRDVSDFTDSVRDPKTSEILDISPCAFGGVS    582
                     |||||||.|:|:|.||||||||::|.||:||||:|.|||||:||:|||||
                 547 TGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVS    596

                 583 VITPGTNASSEVAVLYQDVNCTDVSTAIHADQLTPAWRIYSTGNNVFQTQ    632
                     |||||||.|::|||||||||||:|..|||||||||.||:||||:|||||:
                 597 VITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTR    646

                 633 AGCLIGAEHVDTSYECDIPIGAGICASYHTVS----LLRSTSQKSIVAYT    678
                     ||||||||||:.||||||||||||||||.|.:    ..||.:.:||:|||
                 647 AGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYT    696

                 679 MSLGADSSIAYSNNTIAIPTNFSISITTEVMPVSMAKTSVDCNMYICGDS    728
                     |||||::|:|||||:|||||||:||:|||::||||.||||||.|||||||
                 697 MSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDS    746

                 729 TECANLLLQYGSFCTQLNRALSGIAAEQDRNTREVFAQVKQMYKTPTLKY    778
                     |||:|||||||||||||||||:|||.|||:||:||||||||:||||.:|.
                 747 TECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKD    796

                 779 FGGFNFSQILPDPLKPTKRSFIEDLLFNKVTLADAGFMKQYGECLGDINA    828
                     |||||||||||||.||:||||||||||||||||||||:||||:|||||.|
                 797 FGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAA    846

                 829 RDLICAQKFNGLTVLPPLLTDDMIAAYTAALVSGTATAGWTFGAGAALQI    878
                     |||||||||||||||||||||:|||.||:||::||.|:||||||||||||
                 847 RDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQI    896

                 879 PFAMQMAYRFNGIGVTQNVLYENQKQIANQFNKAISQIQESLTTTSTALG    928
                     |||||||||||||||||||||||||.||||||.||.:||:||::|::|||
                 897 PFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALG    946

                 929 KLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDKVEAEVQIDRL    978
                     ||||||||||||||||||||||||||||||||||||||||||||||||||
                 947 KLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDKVEAEVQIDRL    996

                 979 ITGRLQSLQTYVTQQLIRAAEIRASANLAATKMSECVLGQSKRVDFCGKG   1028
                     ||||||||||||||||||||||||||||||||||||||||||||||||||
                 997 ITGRLQSLQTYVTQQLIRAAEIRASANLAATKMSECVLGQSKRVDFCGKG   1046

                1029 YHLMSFPQAAPHGVVFLHVTYVPSQERNFTTAPAICHEGKAYFPREGVFV   1078
                     ||||||||:||||||||||||||:||:||||||||||:|||:||||||||
                1047 YHLMSFPQSAPHGVVFLHVTYVPAQEKNFTTAPAICHDGKAHFPREGVFV   1096

                1079 FNGTSWFITQRNFFSPQIITTDNTFVSGNCDVVIGIINNTVYDPLQPELD   1128
                     .|||.||:|||||:.|||||||||||||||||||||:|||||||||||||
                1097 SNGTHWFVTQRNFYEPQIITTDNTFVSGNCDVVIGIVNNTVYDPLQPELD   1146

                1129 SFKEELDKYFKNHTSPDVDLGDISGINASVVNIQKEIDRLNEVAKNLNES   1178
                     ||||||||||||||||||||||||||||||||||||||||||||||||||
                1147 SFKEELDKYFKNHTSPDVDLGDISGINASVVNIQKEIDRLNEVAKNLNES   1196

                1179 LIDLQELGKYEQYIKWPWYVWLGFIAGLIAIVMVTILLCCMTSCCSCLKG   1228
                     |||||||||||||||||||:||||||||||||||||:|||||||||||||
                1197 LIDLQELGKYEQYIKWPWYIWLGFIAGLIAIVMVTIMLCCMTSCCSCLKG   1246

                1229 ACSCGSCCKFDEDDSEPVLKGVKLHYT   1255
                     .||||||||||||||||||||||||||
                1247 CCSCGSCCKFDEDDSEPVLKGVKLHYT   1273

# We can see that the algorithm implemented matches with the emboss needle to a certain extent but not completely. This is because emboss needle takes into account extend penalty too.